### Cost of living - items 

In [ ]:
import os
from pandas import concat as pd_concat
from pandas import to_numeric
from pandas import read_csv


# years = [str(year) for year in range(2010, 2021)]
# results_by_year = dict(zip(years, [[] for i in range(len(years))]))

results_by_item = {}
code_mapping = {}

directory = "web_scraper/every_item_by_country_in_usd"
    
for file in os.listdir(directory):
    file_path = directory + '/' + file
    if file_path.endswith('.csv'):
        item_code = file[13:file.index('.')]
        if item_code in results_by_item:
            results_by_item[item_code].append(read_csv(file_path))
        else:
            results_by_item[item_code] = [read_csv(file_path)]
            code_mapping[item_code] = results_by_item[item_code][-1].columns[3]
        results_by_item[item_code][-1].drop(['Unnamed: 0', 'Rank'], axis=1, inplace=True)
        results_by_item[item_code][-1].rename(columns={results_by_item[item_code][-1].columns[1]: file[8:12]}, inplace = True)
        results_by_item[item_code][-1].rename(columns={'City': 'Country'}, inplace = True)
        results_by_item[item_code][-1].set_index('Country', inplace=True)
        results_by_item[item_code][-1] = results_by_item[item_code][-1].apply(lambda x: to_numeric(x, errors='coerce'))

for key in results_by_item:
    results_by_item[key] = pd_concat(results_by_item[key], axis=1, join='outer')
    results_by_item[key] = results_by_item[key].loc[:,~results_by_item[key].columns.duplicated()]

results_by_item['1']

In [ ]:
for key in results_by_item:
    for index, row in results_by_item[key].iterrows():
        if row.isnull().sum() > 0:
            results_by_item[key].drop(index, inplace=True)

In [ ]:
results_by_item['1'].pct_change(axis='columns').max().max()

In [ ]:
print(code_mapping)

In [ ]:
import numpy as np

for key in results_by_item:
    max_val = results_by_item[key].pct_change(axis='columns').max().max()
    if not np.isnan(max_val):
        print(code_mapping[key])
        print('    ', max_val)

In [ ]:
results_by_item['60'].pct_change(axis='columns').max(axis=0)
results_by_item['60'].pct_change(axis='columns').max(axis=1)
#wtf vietnam 2010/2011?

### Cost of living

In [ ]:
from pandas import read_csv
filename = f'./dataset/cost_of_living/cost_of_living.csv'
df = read_csv(filename)

In [ ]:
import numpy as np
for index, row in df.iterrows():
    #print(index, row.isnull().sum(), np.isnan(row['2010']), row['2020'])
    if row.isnull().sum() > 1 or np.isnan(row['2010']) or np.isnan(row['2020']):
        #print(index)
        df.drop(index, inplace=True)

df

In [ ]:
df[~df[[str(x) for x in range(2010, 2021)]].notnull().all(1)]

In [ ]:
df.at[56, '2011'] = (df.at[56, '2010']+df.at[56, '2012'])/2
df.at[82, '2012'] = (df.at[82, '2011']+df.at[82, '2013'])/2
df.at[82, '2012']

In [ ]:
df[~df[[str(x) for x in range(2010, 2021)]].notnull().all(1)]

In [ ]:
df_plt = df

In [ ]:
df_plt.set_index('Country')

In [ ]:
df_plt = df_plt.set_index('Country')
df_plt

In [ ]:
df_plt.T.plot(figsize=(24,12))

In [ ]:
df_plt.pct_change(axis='columns').T.plot(figsize=(24,12))

In [ ]:
filename = f'./dataset/cost_of_living/cost_of_living_after_data_wrangling.csv'
with open(filename, 'w') as file:
    # BE SURE YOU WANT TO OVERWRITE
    #df.to_csv(file, index=False, line_terminator='\n')